### Transfer learning

Install and import packages

In [ ]:
%pip install numpy
%pip install matplotlib
%pip install seaborn
%pip install torch
%pip install torchvision
%pip install helper

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch as th
from torch.utils.data import DataLoader
import torchvision as tv
from torchvision import transforms
import os

### Load dataset

In [ ]:
imagenet_data = tv.datasets.ImageFolder('images/')

In [2]:
files = os.listdir("./images/")

FileNotFoundError: [WinError 3] Det går inte att hitta sökvägen: './images/data'

In [ ]:
def getPaths():
    files = os.listdir("./images/")
    return files

    
def initDataset():
    transform = transforms.Compose([transforms.Resize(255),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor()])
    
    imagenet_data = tv.datasets.ImageFolder('./data/', transform)
    print(imagenet_data)
    
    data_loader = th.utils.data.DataLoader(imagenet_data,
                                            batch_size=32,
                                            shuffle=True,
                                            num_workers=4)
    
    return data_loader

data_loader = initDataset()

### Show images

In [ ]:
def test(iter):
    for image, labels in iter:
        # plt.imshow(image[0].permute(1, 2, 0))
        # plt.show()
        print(labels[0])
imgIterator = iter(data_loader)
test(imgIterator)

### Initialize model

In [ ]:
model = tv.models.resnet18(progress = True)